In [ ]:
from types import SimpleNamespace
from collections import namedtuple
from itertools import starmap
import csv

import jinja2

import ipywidgets
from ipywidgets import Label, HBox, VBox, Textarea, Button, Valid, Tab, HTML
import traitlets

import db
db.use_db()

In [ ]:
def validate_csv(fmt, data, validator=lambda x: True):
    Fields = namedtuple('Fields', fmt)
    for entry in csv.reader(data.split('\n')):
        if not entry:
            continue
        stripped_entry = [e.strip() for e in entry if e.strip()]
        if len(stripped_entry) != len(fmt):
            raise ValueError('expecting "{}" but got: "{}"'
                             .format(", ".join(fmt), ", ".join(entry)))
        validator(stripped_entry)
        yield Fields(*stripped_entry)

In [ ]:
def upload_box():
    upload = SimpleNamespace()
    upload.content = Textarea()
    upload.summary = HTML()
    upload.submit = Button()
    upload.valid = Valid()
    upload.status = Label()
    upload.box = HBox([VBox([upload.content,
                            HBox([upload.submit, upload.status, upload.valid])]),
                       upload.summary,
                      ])

    upload.valid.layout.display = 'none'
    upload.valid.layout.margin = "0 10px"
    #upload.content.width = "50%"
    #upload.content.height = "300px"
    #upload.summary.width = "50%"
    
    return upload

In [ ]:
graders = upload_box()
graders.submit.description = 'Add Graders'
graders.content.placeholder = 'First name, Last Name'

def render_graders():
    template = jinja2.Template('''
    <ul>
    {% for g in graders %}
        <li> {{g.first_name}}, {{g.last_name}} </li>
    {% endfor %}
    </ul>
    ''')
    
    with db.db_session:
        graders.summary.value = template.render(graders=db.Grader.select())

        
@graders.submit.on_click
def _(sender):
    try:
        data = list(validate_csv(['first_name', 'last_name'],  graders.content.value))
    except Exception as e:
        graders.status.value = 'invalid data: {}'.format(str(e))
        graders.valid.value = False
        graders.valid.layout.display = 'inline'
        return
             
    with db.db_session:
        for f in data:
            f = f._asdict()
            if not db.Grader.get(**f):
                db.Grader(**f)
                
    render_graders()
    graders.valid.value = True
    graders.valid.layout.display = 'inline'
    graders.status.value = ''

In [ ]:
students = upload_box()
students.submit.description = 'Add Students'
students.content.placeholder = 'Student ID, First name, Last Name, email address'

def render_students():
    template = jinja2.Template('''
    <ul>
    {% for s in students %}
        <li> {{s.id}}, {{s.first_name}}, {{s.last_name}}, {{s.email}} </li>
    {% endfor %}
    </ul>
    ''')
    
    with db.db_session:
        students.summary.value = template.render(students=db.Student.select())
        

def student_validator(args):
    id, first_name, last_name, email = args
    try:
        id = int(id)
    except Exception:
        raise ValueError('Expected 7-digit student number, but got: {}'.format(id))
    
    if len(str(id)) != 7:
        raise ValueError('Expected 7-digit student number, but got: {}'.format(id))

@students.submit.on_click
def _(sender):
    try:
        data = list(validate_csv(['id', 'first_name', 'last_name', 'email'],
                                 students.content.value, student_validator))
    except Exception as e:
        students.status.value = 'invalid data: {}'.format(str(e))
        students.valid.value = False
        students.valid.layout.display = 'inline'
        return
    
    with db.db_session:
        for f in data:
            if not db.Student.get(id=f.id):
                db.Student(**f._asdict())
                
    render_students()
    students.valid.value = True
    students.valid.layout.display = 'inline'
    students.status.value = ''

In [ ]:

render_graders()
render_students()

content = Tab(children=[graders.box, students.box])
content.set_title(0, 'Add Graders')
content.set_title(1, 'Add Students')
content